In [1]:
import pandas
import sys
import os
import json
sys.path.insert(0, os.getcwd())
from helpers import recursive_find, read_json, create_fedora_results_table

In [2]:
# Read in the experiment json files. One experiment means one package at once version
# across splices (versions of a single dependencny) and predictors
experiments = list(recursive_find("artifacts/results/extracted/fedora", "*.json"))
print('Found %s experiment library files.' % len(experiments))

Found 1402 experiment library files.


In [3]:
df = create_fedora_results_table(experiments)
display(df)

,a,b,original,changed,analysis,seconds,predictor,prediction
0,fedora-libs-35,fedora-libs-36,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,missing-previously-found-symbols,0.00009,symbols,False
1,fedora-libs-35,fedora-libs-36,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,missing-previously-found-exports,0.000008,symbols,True
2,fedora-libs-35,fedora-libs-36,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,abidiff,7.479117,libabigail,False
3,fedora-libs-35,fedora-libs-36,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,abi-compliance-tester,0.356888,abi-laboratory,False
4,fedora-libs-35,fedora-libs-37,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,missing-previously-found-symbols,0.000068,symbols,False
...,...,...,...,...,...,...,...,...
5603,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/32/li...,second/usr/lib/gcc/x86_64-redhat-linux/12/32/l...,abi-compliance-tester,0.108163,abi-laboratory,False
5604,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/32/li...,second/usr/lib/gcc/x86_64-redhat-linux/12/32/l...,missing-previously-found-symbols,0.000004,symbols,Unknown
5605,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/32/li...,second/usr/lib/gcc/x86_64-redhat-linux/12/32/l...,missing-previously-found-exports,0.000002,symbols,Unknown
5606,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/32/li...,second/usr/lib/gcc/x86_64-redhat-linux/12/32/l...,abidiff,0.00576,libabigail,False


In [4]:
# How many unique libraries?
library_count = list(df['original'].values) + list(df['changed'].values)
print('Found %s total libraries' % len(library_count))

Found 11216 total libraries


In [5]:
# Look to see if we have True/False
for predictor in df['predictor'].unique():
    print(predictor)
    print(df[ df['predictor']==predictor]['prediction'].unique())

    
# disagree = df.groupby(["a","b", "original", "changed"])['prediction'].nunique() > 1
# disagree = pandas.DataFrame(disagree)
# Where all predictors agree
all_predictors_agree = pandas.DataFrame(df.groupby(['a', 'b', 'original', 'changed'])['prediction'].nunique() == 1).reset_index()
df_all_agree = df.merge(all_predictors_agree, on=['a', 'b', 'original', 'changed'], suffixes=["", "_all_agree"])
df_all_agree = df_all_agree[df_all_agree['prediction_all_agree']]
df_all_agree

symbols
[False True 'Unknown']
libabigail
[False True]
abi-laboratory
[False True]


,a,b,original,changed,analysis,seconds,predictor,prediction,prediction_all_agree
20,fedora-libs-36,fedora-libs-37,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,missing-previously-found-symbols,0.000069,symbols,True,True
21,fedora-libs-36,fedora-libs-37,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,missing-previously-found-exports,0.00001,symbols,True,True
22,fedora-libs-36,fedora-libs-37,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,abidiff,2.239851,libabigail,True,True
23,fedora-libs-36,fedora-libs-37,first/usr/lib64/LLVMgold.so,second/usr/lib64/LLVMgold.so,abi-compliance-tester,57.164308,abi-laboratory,True,True
24,fedora-libs-34,fedora-libs-36,first/usr/lib64/dri/radeonsi_drv_video.so,second/usr/lib64/dri/radeonsi_drv_video.so,missing-previously-found-symbols,0.000155,symbols,False,True
...,...,...,...,...,...,...,...,...,...
5579,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/plugi...,second/usr/lib/gcc/x86_64-redhat-linux/12/plug...,abi-compliance-tester,284.179451,abi-laboratory,True,True
5580,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/plugi...,second/usr/lib/gcc/x86_64-redhat-linux/12/plug...,missing-previously-found-symbols,0.000036,symbols,True,True
5581,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/plugi...,second/usr/lib/gcc/x86_64-redhat-linux/12/plug...,missing-previously-found-exports,0.000016,symbols,True,True
5582,fedora-libs-36,fedora-libs-37,first/usr/lib/gcc/x86_64-redhat-linux/12/plugi...,second/usr/lib/gcc/x86_64-redhat-linux/12/plug...,abidiff,0.408989,libabigail,True,True


In [6]:
# Convert to wide format where analysis and predictor are columns.
df_wide = pandas.pivot(df, columns=['analysis', 'predictor'], index=['a', 'b', 'original', 'changed'], values='prediction')
df_wide

analysis                                                                                                                            missing-previously-found-symbols  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/usr/lib64/gconv/IBM1161.so                   second/usr/lib64/gconv/IBM1161.so                                              True   
                              first/usr/lib64/gconv/IBM1162.so                   second/usr/lib64/gconv/IBM1162.so                                              True   
                              first/usr/lib64/gconv/IBM1163.so                   second/usr/lib64/gconv/IBM1163.so                                              True   
                              first/usr/lib64/gconv/IBM1164.so                   second/usr/lib64/gconv/IBM1164.so                                              True   
                              first/usr/lib64/gconv/IBM1166.so                   second/usr/lib64/gconv/IBM1166.so                                              True   

analysis                                                                                                                            missing-previously-found-exports  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/11/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/11/32/li...                          Unknown   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/usr/lib64/gconv/IBM1161.so                   second/usr/lib64/gconv/IBM1161.so                     

In [7]:
# Join analysis/predictor columns
df_wide = df_wide.reset_index()
df_wide.columns = ["_".join(names) for names in df_wide.columns]
df_wide.columns

Index(['a_', 'b_', 'original_', 'changed_',
       'missing-previously-found-symbols_symbols',
       'missing-previously-found-exports_symbols', 'abidiff_libabigail',
       'abi-compliance-tester_abi-laboratory'],
      dtype='object')

In [8]:
# Get predictions only
df_predictions = df_wide[['abidiff_libabigail', 'abi-compliance-tester_abi-laboratory',
                         'missing-previously-found-symbols_symbols',
                         'missing-previously-found-exports_symbols']]
df_predictions

,abidiff_libabigail,abi-compliance-tester_abi-laboratory,missing-previously-found-symbols_symbols,missing-previously-found-exports_symbols
0,False,False,Unknown,Unknown
1,False,False,Unknown,Unknown
2,False,False,Unknown,Unknown
3,False,False,Unknown,Unknown
4,False,False,Unknown,Unknown
...,...,...,...,...
1397,True,True,True,True
1398,True,True,True,True
1399,True,True,True,True
1400,True,True,True,True


In [9]:
# Filter to where predictions are either "True" or "False" only.
num_predictors = 4
df_predictions = df_predictions[(df_predictions == True).sum(axis=1) + (df_predictions == False).sum(axis=1) == num_predictors]

# Look to see if we have True/False
for column in df_predictions.columns:
    print(column)
    print(df_predictions[column].unique())
    
df_predictions

abidiff_libabigail
[True False]
abi-compliance-tester_abi-laboratory
[True False]
missing-previously-found-symbols_symbols
[True False]
missing-previously-found-exports_symbols
[True False]


,abidiff_libabigail,abi-compliance-tester_abi-laboratory,missing-previously-found-symbols_symbols,missing-previously-found-exports_symbols
9,True,True,True,True
13,False,False,False,False
14,False,False,False,True
15,True,True,True,True
16,True,True,True,True
...,...,...,...,...
1397,True,True,True,True
1398,True,True,True,True
1399,True,True,True,True
1400,True,True,True,True


In [10]:
# Correlations between predictors.
# The NaN means that the values don't vary (as we see above, the symbols check is always true)
(df_predictions == True).corr()

,abidiff_libabigail,abi-compliance-tester_abi-laboratory,missing-previously-found-symbols_symbols,missing-previously-found-exports_symbols
abidiff_libabigail,1.000000,0.748677,0.720885,0.561932
abi-compliance-tester_abi-laboratory,0.748677,1.000000,0.608812,0.525529
missing-previously-found-symbols_symbols,0.720885,0.608812,1.000000,0.650596
missing-previously-found-exports_symbols,0.561932,0.525529,0.650596,1.000000
